# Project : Customer‑Support Chatbot for an E-Commerce Store

## Roadmap  
We will build a RAG-based chatbot in **six** steps:

1. **Environment setup**
2. **Data preparation**  
   a. Load source documents  
   b. Chunk the text  
3. **Build a retriever**  
   a. Generate embeddings  
   b. Build the FAISS vector index  
4. **Build a generation engine**. Load the *Gemma3-1B* model through Ollama and run a sanity check.  
5. **Build a RAG**. Connect the system prompt, retriever, and LLM together. 
6. **(Optional) Streamlit UI**. Wrap everything in a simple web app so users can chat with the bot.


## 1 - Environment setup

In [1]:
# Import standard libraries for file handling and text processing
import os, pathlib, textwrap, glob

# Load documents from various sources (URLs, text files, PDFs)
from langchain_community.document_loaders import UnstructuredURLLoader, TextLoader, PyPDFLoader

# Split long texts into smaller, manageable chunks for embedding
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Vector store to store and retrieve embeddings efficiently using FAISS
from langchain.vectorstores import FAISS

# Generate text embeddings using OpenAI or Hugging Face models
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings, SentenceTransformerEmbeddings

# Use local LLMs (e.g., via Ollama) for response generation
from langchain.llms import Ollama

# Build a retrieval chain that combines a retriever, a prompt, and an LLM
from langchain.chains import ConversationalRetrievalChain

# Create prompts for the RAG system
from langchain.prompts import PromptTemplate

print("✅ Libraries imported - you're good to go!")

/home/farjam/miniconda3/envs/llm_demo_projects/lib/python3.11/site-packages/requests/__init__.py:113: RequestsDependencyWarning: urllib3 (2.6.3) or chardet (6.0.0.post1)/charset_normalizer (3.4.4) doesn't match a supported version!
  warnings.warn(
/home/farjam/miniconda3/envs/llm_demo_projects/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Libraries imported - you're good to go!


## 2 - Data preparation

In [5]:
pdf_paths = glob.glob("data/Everstorm_*.pdf")

raw_docs = []
for path in pdf_paths:
    raw_docs.extend(PyPDFLoader(path).load())

print(f"Loaded {len(raw_docs)} PDF pages from {len(pdf_paths)} files.")

Ignoring wrong pointing object 81 0 (offset 0)
Ignoring wrong pointing object 80 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)


Loaded 8 PDF pages from 4 files.


In [ ]:
# URLS = [
#     # --- BigCommerce – shipping & refunds ---
#     "https://developer.bigcommerce.com/docs/store-operations/shipping",
#     "https://developer.bigcommerce.com/docs/store-operations/orders/refunds",
#     # --- Stripe – disputes & chargebacks ---
#     # "https://docs.stripe.com/disputes",  
#     # --- WooCommerce – REST API reference ---
#     # "https://woocommerce.github.io/woocommerce-rest-api-docs/v3.html",
# ]

# try:
#     from langchain_community.document_loaders import WebBaseLoader

#     loader = WebBaseLoader(URLS)
#     raw_docs = loader.load()
#     print(f"Fetched {len(raw_docs)} documents from the web.")
# except Exception as e:
#     print("⚠️  Web fetch failed, using offline copies:", e)
#     raw_docs = []

#     from langchain_community.document_loaders import DirectoryLoader, TextLoader

#     # Put any fallback files under ./offline_docs (e.g., .md, .txt, .html)
#     patterns = ["**/*.md", "**/*.txt", "**/*.html"]
#     for pattern in patterns:
#         try:
#             loader = DirectoryLoader(
#                 "offline_docs",
#                 glob=pattern,
#                 loader_cls=TextLoader,
#                 show_progress=True,
#                 use_multithreading=True,
#             )
#             raw_docs.extend(loader.load())
#         except Exception:
#             # Skip unreadable pattern/file types quietly
#             pass

#     print(f"Loaded {len(raw_docs)} offline documents.")

### 2.1 - Chunk the text

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = text_splitter.split_documents(raw_docs)
print(f"✅ {len(chunks)} chunks ready for embedding")

✅ 42 chunks ready for embedding


## 3 -Build a retriever

In [7]:
embeddings = SentenceTransformerEmbeddings(model_name="thenlper/gte-small")

# Single embedding for one sentence
embedding_vector = embeddings.embed_query("What is the capital of France?")
print(len(embedding_vector))

/tmp/ipykernel_9471/34837372.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="thenlper/gte-small")


384


In [8]:
vectordb = FAISS.from_documents(chunks, embeddings)
retriever = vectordb.as_retriever(search_kwargs={"k": 8})

vectordb.save_local("faiss_index")

print("✅ Vector store with", vectordb.index.ntotal, "embeddings")

✅ Vector store with 42 embeddings


## 4 - Build the generation engine

In [10]:
llm = Ollama(model="gemma3:1b", temperature=0.1)
print("✅ LLM loaded:", llm)
print(llm.invoke("What is the capital of France?"))

/tmp/ipykernel_9471/4067377706.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3:1b", temperature=0.1)


✅ LLM loaded: Ollama
Params: {'model': 'gemma3:1b', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': 0.1, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}
The capital of France is **Paris**. 

It’s a very popular and important question! 😊 

Do you want to know more about Paris, like its history or famous landmarks?


## Build a RAG

In [11]:
SYSTEM_TEMPLATE = """
You are a **Customer Support Chatbot**. Use only the information in CONTEXT to answer.
If the answer is not in CONTEXT, respond with “I'm not sure from the docs.”

Rules:
1) Use ONLY the provided <context> to answer.
2) If the answer is not in the context, say: "I don't know based on the retrieved documents."
3) Be concise and accurate. Prefer quoting key phrases from the context.
4) When possible, cite sources as [source: source] using the metadata.

CONTEXT:
{context}

USER:
{question}
"""

In [12]:
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=SYSTEM_TEMPLATE,
)

llm = Ollama(model="gemma3:1b", temperature=0.1)

chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever,
    combine_docs_chain_kwargs={"prompt": prompt},
    return_source_documents=True,
)

In [13]:
test_questions = [
    "If I'm not happy with my purchase, what is your refund policy and how do I start a return?",
    "How long will delivery take for a standard order, and where can I track my package once it ships?",
    "What's the quickest way to contact your support team, and what are your operating hours?",
]

chat_history = []
for q in test_questions:
    result = chain({"question": q, "chat_history": chat_history})
    print("\n❓ Question:", q)
    print("💬 Answer:", result["answer"])
    chat_history.append((q, result["answer"]))
    print("📄 Sources:")
    for doc in result["source_documents"]:
        print(f"   - {doc.metadata['source']}")
    

/tmp/ipykernel_9471/3057405367.py:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain({"question": q, "chat_history": chat_history})



❓ Question: If I'm not happy with my purchase, what is your refund policy and how do I start a return?
💬 Answer: Okay, here’s the refund policy and return process, based on the provided context:

**Refund Policy**

We offer refunds for defective or unsatisfactory items within 30 days of delivery.  Here’s a breakdown:

*   **Warehouse Receipt → Inspection ≤ 3 Business Days:**  If the carrier scans your return label, authorization drops when our returns team receives the original item.
*   **Return Window Exceptions:**
    *   **Holiday Gift Purchases Made 1 Nov – 31 Dec:**  The return period is extended to 31 January.
    *   **“Final Sale” and Custom-Embroidered Items:** Returns are permitted for store credit.
*   **Defect & Warranty Claims (12):**  If you have a defect or warranty claim, you must file it within 12 days of the return.

**How to Start a Return**

1.  **Scan Your Return Label:**  The carrier scans the return label when your item is returned.
2.  **Authorization Drops:**

### 6 - Build the Streamlit UI (optional)

In [19]:
%%bash
cat > app.py <<'PY'
import pathlib, streamlit as st
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import Ollama
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

st.set_page_config(page_title="Customer Support Chatbot", page_icon="🤖")
st.title("Customer Support Chatbot 🤖")

@st.cache_resource
def init_chain():
    vectordb = FAISS.load_local(
        "faiss_index",
        HuggingFaceEmbeddings(model_name="thenlper/gte-small"),
        allow_dangerous_deserialization=True,
    )
    retriever = vectordb.as_retriever(search_kwargs={"k": 8})
    llm = Ollama(model="gemma3:1b", temperature=0.1)

    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
    )

    return ConversationalRetrievalChain.from_llm(
        llm,
        retriever,
        memory=memory,
    )

chain = init_chain()

if "history" not in st.session_state:
    st.session_state.history = []

question = st.chat_input("What is in your mind?")
if question:
    with st.spinner("Thinking..."):
        response = chain(
            {
                "question": question,
                "chat_history": st.session_state.history,   # <- supply it
            }
        )
    st.session_state.history.append((question, response["answer"]))


for user, bot in reversed(st.session_state.history):
    st.markdown(f"**You:** {user}")
    st.markdown(f"**Bot:** {bot}")
PY
echo "✅ Streamlit app code written to app.py"


✅ Streamlit app code written to app.py
